In [3]:
!pip3 install -r llama_requirements.txt

In [4]:
# Import libraries 
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

# Reproducibility
seed = 42
set_seed(seed)

2023-11-20 02:06:14.118765: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 02:06:17.571152: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-20 02:06:17.571172: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-20 02:06:23.604318: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
from huggingface_hub import login
login()

In [21]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

def load_model(model_name="meta-llama/Llama-2-7b-hf", bnb_config=None):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{81920}MB'

    # model = AutoModelForCausalLM.from_pretrained(
    #     model_name,
    #     quantization_config=bnb_config,
    #     device_map="auto", # dispatch efficiently the model on the available ressources
    #     max_memory = {i: max_memory for i in range(n_gpus)},
    # )
    model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
    # tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [22]:
m, t = load_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
!pip3 install peft

In [7]:
get_trainable_params(m)

6738415616

In [8]:
print(m)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [9]:
# m, t = load_model()
# torch.cuda.empty_cache()

from peft import LoraModel, LoraConfig, get_peft_model

lora_config = LoraConfig(task_type="CAUSAL_LM", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)

lora_model = get_peft_model(m, lora_config)


In [6]:
def get_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [1]:
100.0 * get_trainable_params(lora_model) / get_trainable_params(m)

NameError: name 'get_trainable_params' is not defined

In [11]:
lora_model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [7]:
print(lora_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
             

In [24]:
print(m.model.embed_tokens)

Embedding(32000, 4096)


In [29]:
from transformers import LlamaForCausalLM
decoder = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import AutoTokenizer
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [42]:
gpt_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
gpt_tokenizer(["one two three four five six", "check 12"], padding=True)

{'input_ids': [[505, 734, 1115, 1440, 1936, 2237], [9122, 1105, 50257, 50257, 50257, 50257]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 0, 0, 0, 0]]}

In [20]:
import torch
texts = [
    "<s>small one</s>",
    "<s>one two three four five six this is a pretty damn long ass sentence hehe he</s>"
]
llama_tokenizer.add_special_tokens({'pad_token': llama_tokenizer.eos_token})

tokenized_data = llama_tokenizer(
    texts,
    return_tensors='pt', 
    padding=True,
    truncation=True,
    max_length=2048,
    add_special_tokens=False
)
label = torch.where(tokenized_data["attention_mask"] != 0, tokenized_data['input_ids'], -100)
tokenized_data, label

({'input_ids': tensor([[    1,  2319,   697,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2],
         [    1,   697,  1023,  2211,  3023,  5320,  4832,   445,   338,   263,
           5051,  5625, 29876,  1472,  1223, 10541,   540,   354,   540,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 tensor([[    1,  2319,   697,     2,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
         [    1,   697,  1023,  2211,  3023,  5320,  4832,   445,   338,   263,
           5051,  5625, 29876,  1472,  1223, 10541,   540,   354,   540,     2]]))

In [13]:
llama_tokenizer.eos_token

'</s>'

In [4]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer(
    ["We are very happy", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=1000,
    return_tensors="pt",
)

{'input_ids': tensor([[    1,  1334,   526,  1407,  9796, 32000, 32000, 32000, 32000, 32000],
        [    1,  1334,  4966,   366,  1016, 29915, 29873, 26277,   372, 29889]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [50]:
import torch
torch.ones((1,3)).shape

torch.Size([1, 3])

In [14]:
from transformers import ResNetModel
image_repr_model = ResNetModel.from_pretrained("microsoft/resnet-50")

In [55]:
!nvidia-smi

Fri Nov 24 05:06:58 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   73C    P0             290W / 300W |  50132MiB / 81920MiB |     98%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
